In [1]:
import os
import sys
import shutil

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

import matplotlib.patches as patches
import matplotlib

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\n'

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotype_info=pd.read_csv('data/phenotype_pubname.tsv',sep='\t',index_col=0)

In [4]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=phenotypes.columns.difference(['age','sex','cohort','diabetes'])
print(len(binary_continuous_traits))

97


In [7]:
phenotype_info

,phenotype_name_publication,phenotype_category,phenotype_type
phenotype_name,,,
bladder_cancer,Bladder cancer,Cancer,binary
breast_cancer,Breast cancer,Cancer,binary
cervical_cancer,Cervical cancer,Cancer,binary
colorectal_cancer,Colorectal cancer,Cancer,binary
liver_cancer,Liver cancer,Cancer,binary
lung_cancer,Lung cancer,Cancer,binary
prostate_cancer,Prostate cancer,Cancer,binary
stomach_cancer,Stomach cancer,Cancer,binary
thyroid_cancer,Thryroid cancer,Cancer,binary


# marker information

In [44]:
plink_KCHIP_HLA_AA_SNP_1000G_fam

,FID,IID,father,mother,gender,status
0,BHAN000001,BHAN000001,0,0,2,1
1,BHAN000002,BHAN000002,0,0,1,1
2,BHAN000003,BHAN000003,0,0,2,1
3,BHAN000004,BHAN000004,0,0,2,1
4,BHAN000005,BHAN000005,0,0,2,1
5,BHAN000006,BHAN000006,0,0,1,1
6,BHAN000007,BHAN000007,0,0,1,1
7,BHAN000008,BHAN000008,0,0,1,1
8,BHAN000009,BHAN000009,0,0,2,1
9,BHAN000010,BHAN000010,0,0,1,1


In [23]:
plink_KCHIP_HLA_AA_SNP_1000G_bim.head()

,chrom,pos,cm,a1,a2
snp,,,,,
6:28000361_T/C,6,28000361,0,C,T
6:28000610_C/A,6,28000610,0,A,C
6:28001003_A/C,6,28001003,0,C,A
6:28001033_T/C,6,28001033,0,C,T
6:28001610_T/C,6,28001610,0,C,T


In [33]:
marker_HLA=plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].copy()
marker_AA=plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('AA_')].copy()
marker_all=plink_KCHIP_HLA_AA_SNP_1000G_bim[:].copy()

In [38]:
marker_HLA

,chrom,pos,cm,a1,a2
snp,,,,,
HLA_A*01:01,6,29910247,0,p,a
HLA_A*02:01,6,29910247,0,p,a
HLA_A*02:06,6,29910247,0,p,a
HLA_A*02:07,6,29910247,0,p,a
HLA_A*03:01,6,29910247,0,p,a
HLA_A*11:01,6,29910247,0,p,a
HLA_A*24:02,6,29910247,0,p,a
HLA_A*26:01,6,29910247,0,p,a
HLA_A*26:02,6,29910247,0,p,a


In [30]:
marker_HLA[~marker_HLA.duplicated('pos')]#,marker_HLA[~marker_HLA.duplicated('pos')].shape

,chrom,pos,cm,a1,a2
snp,,,,,
HLA_A*01:01,6,29910247,0,p,a
HLA_C*01:02,6,31236526,0,p,a
HLA_B*07:02,6,31321649,0,p,a
HLA_DRB1*01:01,6,32546547,0,p,a
HLA_DQA1*01:01,6,32605183,0,p,a
HLA_DQB1*02:01,6,32627241,0,p,a
HLA_DPA1*01:03,6,33032346,0,p,a
HLA_DPB1*02:01,6,33043703,0,p,a


In [32]:
marker_AA[~marker_AA.duplicated('pos')]#,marker_HLA[~marker_HLA.duplicated('pos')].shape

,chrom,pos,cm,a1,a2
snp,,,,,
AA_A_-15_29910359_exon1,6,29910359,0,L,V
AA_A_-11_29910371_exon1,6,29910371,0,L,S
AA_A_9_29910558_exon2_F,6,29910558,0,p,a
AA_A_17_29910582_exon2,6,29910582,0,S,R
AA_A_44_29910663_exon2,6,29910663,0,K,R
AA_A_56_29910699_exon2,6,29910699,0,R,G
AA_A_62_29910717_exon2_E,6,29910717,0,p,a
AA_A_63_29910720_exon2_E,6,29910720,0,p,a
AA_A_65_29910726_exon2,6,29910726,0,G,R


In [37]:
#marker_all[marker_all['pos']>=33053550]

In [42]:
#marker_all[~marker_all.duplicated('pos')]#,marker_HLA[~marker_HLA.duplicated('pos')].shape
marker_all.loc[marker_all.index.difference(marker_HLA.index).difference(marker_AA.index)]

,chrom,pos,cm,a1,a2
snp,,,,,
6:28000361_T/C,6,28000361,0,C,T
6:28000610_C/A,6,28000610,0,A,C
6:28001003_A/C,6,28001003,0,C,A
6:28001033_T/C,6,28001033,0,C,T
6:28001610_T/C,6,28001610,0,C,T
6:28002045_G/A,6,28002045,0,A,G
6:28002139_G/A,6,28002139,0,A,G
6:28002388_G/A,6,28002388,0,A,G
6:28002818_C/T,6,28002818,0,T,C


In [21]:
len(plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')]['pos'].unique()),\
len(plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('AA_')]['pos'].unique()),\
len(plink_KCHIP_HLA_AA_SNP_1000G_bim['pos'].unique())

(8, 349, 67448)

# MAF

In [8]:
def cal_maf(dosage_array):
    maf=(1*(dosage_array==1).sum()+2*(dosage_array==2).sum())/\
         (2*(dosage_array!=-1).sum())
    return maf

maf_list_list=[]
for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].iterrows()):
    maf_list=[]
    assert row['a1']=='p'
    dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP)
    maf_list.append(cal_maf(dosage))
    for cohort in range(1,3+1):
        cohort_dosage=dosage[phenotypes['cohort']==cohort]
        #print('total: {:5d} -1:{:5d} 0:{:5d} 1:{:5d} 2:{:5d}'.format(len(cohort_dosage),(cohort_dosage==-1).sum(),(cohort_dosage==0).sum(),(cohort_dosage==1).sum(),(cohort_dosage==2).sum()))
        maf_list.append(cal_maf(cohort_dosage))
    
    maf_list_list.append(maf_list)

In [9]:
assert (phenotypes.index!=plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']).sum()==0

In [10]:
maf_df=pd.DataFrame(maf_list_list,index=plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index,columns=['all','AS','CT','NC'])

In [11]:
maf_df.index=maf_df.index.str.replace('_','-')

In [13]:
maf_df.head()

,all,AS,CT,NC
snp,,,,
HLA-A*01:01,0.017462,0.018733,0.017275,0.017930
HLA-A*02:01,0.164765,0.164454,0.163066,0.173798
HLA-A*02:06,0.086112,0.083081,0.086044,0.087692
HLA-A*02:07,0.034538,0.037465,0.034419,0.033982
HLA-A*03:01,0.015986,0.017878,0.015576,0.017375


In [14]:
maf_df.to_csv(data_out_pub_path+'HLA_allele_freq.tsv',sep='\t')

# GWAS each phenotype

In [19]:
for phenotype_name in binary_continuous_traits:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    
    pheotype_name_publication=phenotype_info.loc[phenotype_name]['phenotype_name_publication'].replace(' ','_').replace('/','_')
    
    data_out_pub_phenotype_path=data_out_pub_path+pheotype_name_publication+'/'
    pathlib.Path(data_out_pub_phenotype_path).mkdir(parents=True, exist_ok=True)
    
    
    
    #pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    #phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'
    #phenotype_type    
    
    for step_idx_sub in range(1,100):
        print("phenotype_name: {}, phenotype_type:{} , Step : {} ".format(phenotype_name,'  ',step_idx_sub))
        #print(step_idx_sub)
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break
            
        result_merge=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t',index_col=0)
        result_merge=result_merge.drop(columns=['phenotype_name'])  
        result_merge['phenotype_name_publication']=phenotype_info.loc[phenotype_name]['phenotype_name_publication']
        #print(result_merge.columns)
        
        result_merge[['phenotype_name_publication','phenotype_type', 'samples(case/control)', 'step', 'condition',
               'marker_name', 'marker_name_pub', 'note', 'term', 'POS', 'A1', 'A2',
               'A1_freq_case', 'A1_freq_control', 'multi_allele', 'nobs', 'Z', 'coef',
               'std', 'chisq', 'df', 'P']].to_csv(data_out_pub_phenotype_path+'step_{:02d}.result.tsv'.format(step_idx_sub),sep='\t',index=None)        
        
        shutil.copy(data_out_assoc_phenotype_path+'step_{:02d}.merge.manhattan.png'.format(step_idx_sub),
                    data_out_pub_phenotype_path+'step_{:02d}.manhattan.png'.format(step_idx_sub))         
        
        
        #break
    #break
        

phenotype_name: FEV_over_FVC_predicted, phenotype_type:   , Step : 1 
phenotype_name: FEV_over_FVC_predicted, phenotype_type:   , Step : 2 
meets end 2
phenotype_name: FEV_predicted, phenotype_type:   , Step : 1 
phenotype_name: FEV_predicted, phenotype_type:   , Step : 2 
phenotype_name: FEV_predicted, phenotype_type:   , Step : 3 
meets end 3
phenotype_name: FVC_predicted, phenotype_type:   , Step : 1 
phenotype_name: FVC_predicted, phenotype_type:   , Step : 2 
phenotype_name: FVC_predicted, phenotype_type:   , Step : 3 
meets end 3
phenotype_name: MVV_predicted, phenotype_type:   , Step : 1 
phenotype_name: MVV_predicted, phenotype_type:   , Step : 2 
meets end 2
phenotype_name: acute_liver_disease, phenotype_type:   , Step : 1 
phenotype_name: acute_liver_disease, phenotype_type:   , Step : 2 
meets end 2
phenotype_name: albumin, phenotype_type:   , Step : 1 
phenotype_name: albumin, phenotype_type:   , Step : 2 
phenotype_name: albumin, phenotype_type:   , Step : 3 
phenotype_nam

phenotype_name: hemoglobin, phenotype_type:   , Step : 2 
phenotype_name: hemoglobin, phenotype_type:   , Step : 3 
meets end 3
phenotype_name: hip_circumference, phenotype_type:   , Step : 1 
phenotype_name: hip_circumference, phenotype_type:   , Step : 2 
phenotype_name: hip_circumference, phenotype_type:   , Step : 3 
phenotype_name: hip_circumference, phenotype_type:   , Step : 4 
phenotype_name: hip_circumference, phenotype_type:   , Step : 5 
meets end 5
phenotype_name: hscrp, phenotype_type:   , Step : 1 
phenotype_name: hscrp, phenotype_type:   , Step : 2 
phenotype_name: hscrp, phenotype_type:   , Step : 3 
meets end 3
phenotype_name: hyperlipidemia, phenotype_type:   , Step : 1 
phenotype_name: hyperlipidemia, phenotype_type:   , Step : 2 
phenotype_name: hyperlipidemia, phenotype_type:   , Step : 3 
meets end 3
phenotype_name: hypertension, phenotype_type:   , Step : 1 
phenotype_name: hypertension, phenotype_type:   , Step : 2 
phenotype_name: hypertension, phenotype_type: 

phenotype_name: wbc, phenotype_type:   , Step : 3 
phenotype_name: wbc, phenotype_type:   , Step : 4 
phenotype_name: wbc, phenotype_type:   , Step : 5 
phenotype_name: wbc, phenotype_type:   , Step : 6 
meets end 6
phenotype_name: weight, phenotype_type:   , Step : 1 
phenotype_name: weight, phenotype_type:   , Step : 2 
phenotype_name: weight, phenotype_type:   , Step : 3 
phenotype_name: weight, phenotype_type:   , Step : 4 
phenotype_name: weight, phenotype_type:   , Step : 5 
phenotype_name: weight, phenotype_type:   , Step : 6 
meets end 6


In [17]:
result_merge

,phenotype_type,samples(case/control),step,condition,marker_name,marker_name_pub,note,term,POS,A1,...,A1_freq_control,multi_allele,nobs,Z,coef,std,chisq,df,P,phenotype_name_publication
0,continuous,7440,1,NaN,6:28000361_T/C,rs1529749,unphased bialleic,ADD,28000361,C,...,NaN,NaN,7440,1.323770,0.023202,0.017527,NaN,NaN,0.185621,FEV/FVC
7,continuous,7440,1,NaN,6:28000610_C/A,rs17765491,unphased bialleic,ADD,28000610,A,...,NaN,NaN,7440,1.295790,0.022717,0.017531,NaN,NaN,0.195089,FEV/FVC
14,continuous,7440,1,NaN,6:28001003_A/C,rs17708949,unphased bialleic,ADD,28001003,C,...,NaN,NaN,7440,1.399010,0.025444,0.018187,NaN,NaN,0.161852,FEV/FVC
21,continuous,7440,1,NaN,6:28001033_T/C,rs149941,unphased bialleic,ADD,28001033,C,...,NaN,NaN,7440,1.377710,0.015196,0.011030,NaN,NaN,0.168333,FEV/FVC
28,continuous,7440,1,NaN,6:28001610_T/C,rs149942,unphased bialleic,ADD,28001610,C,...,NaN,NaN,7440,1.377710,0.015196,0.011030,NaN,NaN,0.168333,FEV/FVC
35,continuous,7440,1,NaN,6:28002045_G/A,rs9393873,unphased bialleic,ADD,28002045,A,...,NaN,NaN,7440,1.175910,0.020529,0.017458,NaN,NaN,0.239668,FEV/FVC
42,continuous,7440,1,NaN,6:28002139_G/A,rs77234126,unphased bialleic,ADD,28002139,A,...,NaN,NaN,7440,1.691470,0.026379,0.015595,NaN,NaN,0.090789,FEV/FVC
49,continuous,7440,1,NaN,6:28002388_G/A,rs149943,unphased bialleic,ADD,28002388,A,...,NaN,NaN,7440,1.379620,0.020578,0.014916,NaN,NaN,0.167746,FEV/FVC
56,continuous,7440,1,NaN,6:28002818_C/T,rs149944,unphased bialleic,ADD,28002818,T,...,NaN,NaN,7440,1.377710,0.015196,0.011030,NaN,NaN,0.168333,FEV/FVC
63,continuous,7440,1,NaN,6:28002853_T/C,rs149945,unphased bialleic,ADD,28002853,C,...,NaN,NaN,7440,1.377710,0.015196,0.011030,NaN,NaN,0.168333,FEV/FVC
